## AI Agent智能应用从0到1定制开发 
## AI Agent Intelligent Application Custom Development from 0 to 1
******
- 此代码为网课《AI Agent智能应用从0到1定制开发》的配套代码，需要注意本套代码建议与网课适配配合食用。
- This code for the online course <AI Agent Intelligent Applications from 0 to 1 custom development> supporting code, need to pay attention to this set of code is recommended with the online course adapted to work with consumption.
- 需要注意由于课程开发周期的原因，langchain版本跨越了3个大版本，部分代码会与视频演示有差别!
- Note that due to the course development cycle, the langchain version spans 3 major releases and some of the code will differ from the video demo!
- 课程地址：https://coding.imooc.com/class/822.html
- Course address: https://coding.imooc.com/class/822.html

### 从环境变量中读取密钥
### Read the key from the environment variable
- 注意：尽量将你的OpenAI Key存储在类似.env文件中，而不是明文暴露在代码里，这是一种基本的安全措施
- Note: Try to store your OpenAI Key in something like an .env file, rather than exposing it explicitly in code, as a basic safety measure!
******

In [ ]:

import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("asset/openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")
os.environ["ELEVEN_API_KEY"] = os.getenv("ELEVEN_API_KEY")
os.environ["AZURE_COGS_KEY"] = os.getenv("AZURE_COGS_KEY")
os.environ["AZURE_COGS_ENDPOINT"] = os.getenv("AZURE_COGS_ENDPOINT")
os.environ["AZURE_COGS_REGION"] = os.getenv("AZURE_COGS_REGION")

### 最佳开发实践
### Best development practices
<hr>

- 本地大模型调用
- 需要开启ollama本地服务器
- Local Large Model Calling
- Need to enable ollama local server

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2-chinese:7b-chat-q4_0")
llm.invoke(input="你好啊")

In [ ]:
#使用流式
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Ollama(
    model="llama2-chinese:7b-chat-q4_0", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)
llm.invoke(input="第一个登上月球的人是谁?")

- 模型评估
- Model evaluation

In [ ]:
from langchain_openai import OpenAI
from langchain.evaluation import load_evaluator
llm = OpenAI()
evaluator = load_evaluator("criteria", llm=llm, criteria="conciseness")
eval_result = evaluator.evaluate_strings(
    prediction="four.",
    input="What's 2+2?",
)
print(eval_result)

In [ ]:
#配置要评测的模型1
from langchain.evaluation import load_evaluator
from langchain_community.llms.chatglm import ChatGLM

#注意：这里的endpoint_url是GPUHub的地址，不是OpenAI的地址
llm = ChatGLM(
    endpoint_url="https://u41510-8a66-bea62533.westc.gpuhub.com:8443/",
    #top_p=0.9,
    history=[],
    max_token=80000,
    model_kwargs={"sample_model_args": False},
)
#内置标准：回答是否简明，criteria标准，conciseness简明度
evaluator = load_evaluator("criteria", llm=llm, criteria="conciseness")


In [ ]:
#inpt 输入的评测问题
#prediction 预测的答案
# 返回值 Y/N 是否符合
# 返回值score 1-0分数，1为完全，0为不完全
eval_result = evaluator.evaluate_strings(
    prediction="four.",
    input="What's 2+2?",
)
print(eval_result)

In [ ]:
! pip install text_generation

In [ ]:
#需要评测的模型2 本地模型
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama2-chinese")
evaluator = load_evaluator("criteria", llm=llm, criteria="conciseness")

In [ ]:
#inpt 输入的评测问题
#prediction 预测的答案
# 返回值 Y或者N是否符合
# 返回值score 1-0分数，1为完全，0为不完全
eval_result = evaluator.evaluate_strings(
    prediction="What's 2+2? That's an elementary question. The answer you're looking for is that two and two is four.",
    input="What's 2+2?",
)
print(eval_result)

In [5]:
# 内置的一些评估标准
from langchain.evaluation import Criteria

list(Criteria)

[<Criteria.CONCISENESS: 'conciseness'>,
 <Criteria.RELEVANCE: 'relevance'>,
 <Criteria.CORRECTNESS: 'correctness'>,
 <Criteria.COHERENCE: 'coherence'>,
 <Criteria.HARMFULNESS: 'harmfulness'>,
 <Criteria.MALICIOUSNESS: 'maliciousness'>,
 <Criteria.HELPFULNESS: 'helpfulness'>,
 <Criteria.CONTROVERSIALITY: 'controversiality'>,
 <Criteria.MISOGYNY: 'misogyny'>,
 <Criteria.CRIMINALITY: 'criminality'>,
 <Criteria.INSENSITIVITY: 'insensitivity'>,
 <Criteria.DEPTH: 'depth'>,
 <Criteria.CREATIVITY: 'creativity'>,
 <Criteria.DETAIL: 'detail'>]

In [ ]:
#模型1
#from langchain_community.llms.chatglm import ChatGLM

#llm = ChatGLM(
#    endpoint_url="https://u41510-8a66-bea62533.westc.gpuhub.com:8443/",
#    top_p=0.9,
#    history=[
#    ],
#    max_token=80000,
#    model_kwargs={"sample_model_args": False},
#)
#模型2
#from langchain_community.chat_models import ChatOllama
#llm = ChatOllama(model="llama2-chinese")
from langchain_openai import OpenAI
llm = OpenAI()
#使用enum格式加载标准
from langchain.evaluation import EvaluatorType
#自定义评估标准
custom_criterion = {
    "幽默性": "输出的内容是否足够幽默或者包含幽默元素",
}
eval_chain = load_evaluator(
    EvaluatorType.CRITERIA,
    llm=llm,
    criteria=custom_criterion,
)
query = "给我讲一个笑话"
prediction = "有一天，小明去买菜，结果买了一堆菜回家，结果发现自己忘了带钱。"
eval_result = eval_chain.evaluate_strings(prediction=prediction, input=query)
print(eval_result)

- 模型比较
- model comparison

In [ ]:
from langchain.model_laboratory import ModelLaboratory
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain_community.llms.chatglm import ChatGLM
from langchain_community.chat_models import ChatOllama

In [ ]:
#比较openai、ChatGLM、ChatOllama三个模型的效果
llms = [
    OpenAI(temperature=0),
    ChatGLM(endpoint_url="https://u41510-8a66-bea62533.westc.gpuhub.com:8443/",history=[],),
    ChatOllama(model="llama2-chinese"),
]

In [ ]:
model_lab = ModelLaboratory.from_llms(llms)

In [ ]:
model_lab.compare("齐天大圣的师傅是谁？")

In [ ]:
from langchain import hub
from langchain.prompts import PromptTemplate
from langchain.chains import  LLMChain
prompt = PromptTemplate(
    template="给我讲一个关于{topic}的笑话，控制在30个字之内",
    input_variables=["topic"],
)
open_ai_llm = OpenAI(temperature=0)
chatglm_llm =ChatGLM(endpoint_url="https://u41510-8a66-bea62533.westc.gpuhub.com:8443/",history=[],)
openai_chain = LLMChain(
    llm=open_ai_llm,
    prompt=prompt,
    verbose=True,
)
chatglm_chain = LLMChain(
    llm=chatglm_llm,
    prompt=prompt,
    verbose=True,
)


In [ ]:
chains = [openai_chain, chatglm_chain]
names = [str(open_ai_llm), str(chatglm_llm)]

In [ ]:
model_lab = ModelLaboratory(chains, names=names)

In [ ]:
model_lab.compare("写作业")

### 调试
### debug

In [6]:
#创建LLM 
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

In [7]:
#定义agent的prompt
#https://smith.langchain.com/hub/hwchase17/openai-functions-agent
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [8]:
#定义工具,加载预制的工具,注意有的工具需要提供LLM
from langchain.agents import load_tools

tools = load_tools(["llm-math"], llm=llm)
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Comp

In [9]:
#创建agent
from langchain.agents import create_openai_functions_agent
agent = create_openai_functions_agent(llm, tools, prompt)

In [10]:
#定义agent的执行器，这里注意与老版本的不同
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [11]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

- set_debug 

In [12]:
from langchain.globals import set_debug

set_debug(True)
agent_executor.invoke({"input": "你好啊！"})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "你好啊！"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": []
}
[chain/end] [chain:A

{'input': '你好啊！', 'output': '你好！有什么可以帮助你的吗？'}

- set_verbose

In [13]:
from langchain.globals import set_verbose

set_verbose(True)
agent_executor.invoke({"input": "你好啊！"})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "你好啊！"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": []
}
[chain/end] [chain:A

{'input': '你好啊！', 'output': '你好！有什么可以帮助你的吗？'}